In [1]:
import autosklearn.classification
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import sklearn.metrics

In [3]:
train = pd.read_csv('../Datasets/SPECT Heart/spect_train_binary.csv')
test = pd.read_csv('../Datasets/SPECT Heart/spect_test_binary.csv')

In [8]:
X_train = train.drop('OVERALL_DIAGNOSIS', axis=1)
y_train = train['OVERALL_DIAGNOSIS']
X_test = test.drop('OVERALL_DIAGNOSIS', axis=1)
y_test = test['OVERALL_DIAGNOSIS']


In [9]:
automl = autosklearn.classification.AutoSklearnClassifier()
automl.fit(X_train, y_train)
y_hat = automl.predict(X_test)

In [10]:
sklearn.metrics.accuracy_score(y_test, y_hat)

0.8128342245989305

In [11]:
import pickle
with open('./models/spect_heart_automl.pkl', 'wb') as f:
    pickle.dump(automl, f)